# Project description

The aim of this project is to build machine learning classifiers to be used on the [Adult Dataset](https://archive.ics.uci.edu/ml/datasets/adult). kNN, random forest, classification trees and a Muli-Layer Perceptron model will be trained to predict whether a person income is more than US$50k/year or not. The accuracy, precision and recall for each classifier will be compared.

The Adult Dataset consists of 48842 observations (with missing data) and 14 features. The UCI Repository already provides files for the training and testing sets (2/3 and 1/3 of total observations, respectively).

## Problem

Practice: Classifiers for the Adult Dataset

1. Get the dataset Adult (either in the UCI Repository or in Kaggle).

2. Analyze features, missing data, and overall characteristics of the dataset (using tools in sklearn). Select features, discretize numerical features, handle missing data. 

3. Build a kNN classifier to predict class variable ">50k" (that is, whether person gets more than US$50k/year or not). Select the hyperparameter using cross-validation; test the final classifier using a holdout dataset.

4. Repeat (3) using either a classification tree or a random forest.

5. **Present accuracy, precision and recall for the best classifier in (3) and in (4).**

6. Extra points: repeat (3) using a Multi-Layer Perceptron. 

# Exploratory analysis

In [88]:
# Loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


#from skleaerrn import metrics
#from pandas.tools.plotting import scatter_matrix
#import matplotlib.pyplot as plt
#from sklearn import model_selection
#from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from sklearn.naive_bayes import GaussianNB
#from sklearn import preprocessing
#from sklearn.svm import SVC
#from sklearn.cross_validation import train_test_split
#from sklearn import tree

In [89]:
#Loading the test dataset
df_train = pd.read_csv('./dataset/adult_data.csv', header=None)

In [90]:
df_train.shape #32561 observations and 14 features 

(32561, 15)

In [91]:
df_train.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [92]:
#Loading the training dataset
df_test = pd.read_csv('./dataset/adult_test.csv', header=None)

In [93]:
df_test.shape #16281 observations and 14 features

(16281, 15)

In [94]:
df_test.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [95]:
#The dataset will be splited into training and test after the preprocessing step
df = pd.concat([df_train, df_test])
df.shape

(48842, 15)

In [96]:
#Change colnames
df = df.rename(columns={
    0:"age",
    1:"workclass",
    2:"fnlwgt",
    3:"education",
    4:"education-num",
    5:"matrial-status",
    6:"occupation",
    7:"relationship",
    8:"race",
    9:"sex",
    10:"capital-gain",
    11:"capital-loss",
    12:"hours-per-week",
    13:"native-country",
    14:"income"
})

In [97]:
#df.select_dtypes(include="object").apply(pd.unique)

In [98]:
#Verify the class of each feature
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
matrial-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

In [103]:
#Verify the levels of categorial data and respectively frequency
[print(f"\n## {column} ##\n{df[column].value_counts(normalize=True)*100}") for column in df.select_dtypes(include=['object'])]


## workclass ##
 Private             69.419762
 Self-emp-not-inc     7.907129
 Local-gov            6.420703
 ?                    5.730724
 State-gov            4.055935
 Self-emp-inc         3.470374
 Federal-gov          2.931903
 Without-pay          0.042996
 Never-worked         0.020474
Name: workclass, dtype: float64

## education ##
 HS-grad         32.316449
 Some-college    22.271815
 Bachelors       16.430531
 Masters          5.439990
 Assoc-voc        4.219729
 11th             3.709922
 Assoc-acdm       3.277917
 10th             2.843864
 7th-8th          1.955284
 Prof-school      1.707547
 9th              1.547848
 12th             1.345154
 Doctorate        1.216166
 5th-6th          1.042136
 1st-4th          0.505712
 Preschool        0.169936
Name: education, dtype: float64

## matrial-status ##
 Married-civ-spouse       45.819172
 Never-married            32.998239
 Divorced                 13.580525
 Separated                 3.132550
 Widowed                 

[None, None, None, None, None, None, None, None, None]

In [100]:
#Checking for missing data reported as NA
df.isna().any()

age               False
workclass         False
fnlwgt            False
education         False
education-num     False
matrial-status    False
occupation        False
relationship      False
race              False
sex               False
capital-gain      False
capital-loss      False
hours-per-week    False
native-country    False
income            False
dtype: bool

In [107]:
# Remove observations with unknown values (reported as "?")
df['native-country'].replace("?", np.NaN)

0         United-States
1         United-States
2         United-States
3         United-States
4                  Cuba
              ...      
16276     United-States
16277     United-States
16278     United-States
16279     United-States
16280     United-States
Name: native-country, Length: 48842, dtype: object

# Preprocessing data